# Database Creation in Python

## Outline

* Database engines and the `sqlalchemy` abstractions
* Creating a row class
* Adding data to a database table
* Basic queries

## Types of databases

* Relational
* NoSQL

See [this blog post](https://www.alooma.com/blog/types-of-modern-databases) for a nice overview

## Some common relational/SQL engines

* `sqlite` (comes with Python)
* `postgres`
* `MySQL`

## Types of NoSQL databased

* [Key/Value stores](https://en.wikipedia.org/wiki/Key-value_database)
* [Wide Column stores](https://en.wikipedia.org/wiki/Wide_column_store)
* [Document stores](https://en.wikipedia.org/wiki/Document-oriented_database)

## What we will use

* `sqllite` SQL database engine
* `sqlalchemy` object abstraction

## What is `sqlalchemy`?

* Everything in Python is an object
* `sqlalchemy` allows performing SQL with a dot-chain
    * `session.query(Address).filter(Address.person == person).all()`
* provides classes/objects for
    * Tables
    * Rows
    * Metadata
    * Inspections

In [38]:
import sqlalchemy
sqlalchemy.__version__ 

'1.2.7'

# Case Study - Super Hero Traits

The file `./data/heroes_information.csv` contains information an various comic book super heros.  Let's read this file using `csv.Dictreader`

## Creating a database with `pandas`

We use `pandas` to

* Read the data
* Clean the column names and `NA`s
* Create the `sqlite` database

## Step 1 - Read and Clean the Data

First step is to

* Inspect the data
* Read the data, paying attention to `NA`s
* Clean up the column names

## Inspecting the raw csv file

In [39]:
!head  ./data/heroes_information.csv

Id,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441.0
1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441.0
4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0
5,Absorbing Man,Male,blue,Human,No Hair,193.0,Marvel Comics,-,bad,122.0
6,Adam Monroe,Male,blue,-,Blond,-99.0,NBC - Heroes,-,good,-99.0
7,Adam Strange,Male,blue,Human,Blond,185.0,DC Comics,-,good,88.0
8,Agent 13,Female,blue,-,Blond,173.0,Marvel Comics,-,good,61.0


## Be a Data Detective!

<img src="./img/heroes_problem_1.png" width=600>

## Be a Data Detective!

<img src="./img/heroes_problem_2.png" width=600>

## Read the file into `pandas`

In [40]:
import pandas as pd
df = pd.read_csv('./data/heroes_information.csv', na_values=['-', '', '-99.0'])
df.head()

,Id,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,NaN,good,441.0
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,NaN,bad,441.0
4,4,Abraxas,Male,blue,Cosmic Entity,Black,NaN,Marvel Comics,NaN,bad,NaN


## Cleaning up the column names

First, create a `dict` of new-names/old-names.

In [41]:
from dfply import *
clean_names = lambda name: name.strip().lower().replace(' ', '_')
new_names = {clean_names(old_name):old_name for old_name in df.columns}
new_names

{'id': 'Id',
 'name': 'name',
 'gender': 'Gender',
 'eye_color': 'Eye color',
 'race': 'Race',
 'hair_color': 'Hair color',
 'height': 'Height',
 'publisher': 'Publisher',
 'skin_color': 'Skin color',
 'alignment': 'Alignment',
 'weight': 'Weight'}

## Cleaning up the column names

Now us rename to fix the names.

In [42]:
from dfply import rename
df_renamed = (df >>
              rename(**new_names))
df_renamed.columns

Index(['id', 'name', 'gender', 'eye_color', 'race', 'hair_color', 'height',
       'publisher', 'skin_color', 'alignment', 'weight'],
      dtype='object')

## How to create a database with `pandas`

1. Start a `sqlalchemy` engine
2. Define the `sqlalchemy` column types
4. Use `to_sql` to write the database

## Step 1 - Creating a `sqlalchemy` engine

In [43]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///heroes.db', echo=False)

## Step 2 - Defining the `sqlalchemy` column types

In [44]:
from sqlalchemy import Integer, Float, String

sql_types = {'id': Integer, 
             'name': String, 
             'gender': String, 
             'eye_color': String, 
             'race': String, 
             'hair_color': String, 
             'height':Float, 
             'publisher': String, 
             'skin_color': String, 
             'alignment': String, 
             'weight':Float}

## Step 3 - Writing the database with `to_sql`

In [45]:
df_renamed.to_sql('heroes', 
                  con=engine, 
                  dtype=sql_types, 
                  index=False,
                  if_exists='replace')

OperationalError: (sqlite3.OperationalError) too many SQL variables [SQL: 'INSERT INTO heroes (id, name, gender, eye_color, race, hair_color, height, publisher, skin_color, alignment, weight) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 

# `sqlalchemy` Queries

## Working with an existing database

1. Create an engine linked to the database file
2. Automatically create Python objects for the tables/columns/etc.
3. Inspect the database
4. Create a session connected to the session

## Step 1 - Connect to a database

`create_engine`

* connects to an existing db
* creates it if necessary
* speaks the database's language

In [46]:
engine2 = create_engine('sqlite:///heroes.db')

## To echo, or not to echo

* In this example, `db.echo = True`
* Normally, `db.echo = False`
    * SHUT UP ALREADY

In [47]:
engine2.echo = False

## Step 2 - Create Python class for the table(s)

To automatically create Python class

1. Use `automap_base` to create a `Base` class.
2. Connect to the `engine` with `reflect=True`
3. Make aliases to class for all tables

#### a) Create the `Base` class

In [48]:
from sqlalchemy.ext.automap import automap_base
Base = automap_base()

#### b) Use reflection to `prepare` the `Base` class

In [49]:
Base.prepare(engine2, reflect=True)

#### c) Create a Python class for each table

In [50]:
Hero = Base.metadata.tables['heroes']
Hero

Table('heroes', MetaData(bind=None), Column('id', INTEGER(), table=<heroes>), Column('name', VARCHAR(), table=<heroes>), Column('gender', VARCHAR(), table=<heroes>), Column('eye_color', VARCHAR(), table=<heroes>), Column('race', VARCHAR(), table=<heroes>), Column('hair_color', VARCHAR(), table=<heroes>), Column('height', FLOAT(), table=<heroes>), Column('publisher', VARCHAR(), table=<heroes>), Column('skin_color', VARCHAR(), table=<heroes>), Column('alignment', VARCHAR(), table=<heroes>), Column('weight', FLOAT(), table=<heroes>), schema=None)

## Step 3 - Inspecting the database

We make a `inspect` object that allows us to inspect with the `db`

## Fine-grain Inspection with `inspect`

In [51]:
from sqlalchemy import inspect
insp = inspect(engine2)

## Getting the name of all tables

In [52]:
insp.get_table_names()

['heroes']

## Getting detailed information on columns

In [53]:
insp.get_columns('heroes')

[{'name': 'id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'name',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'gender',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'eye_color',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'race',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'hair_color',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'height',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'publisher',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto

## Step 4 - Making a session object

We make a `session` that allows us to interact with the `db`

#### a) Make a `Session` class bound to `db`

In [54]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine2)

#### b) Make an instance of the `Session` class

In [55]:
session = Session()

## `sqlalchemy` query basics

To make a query, we will

1. Create a `SQL` statement using 
    a. `sqlalchemy.select`
    b. `text` and a raw `SQL` statement string
2. Use the `session` to excute the statement

## Our first query, the Python way 

* `select` allows us to create a statment by dot chaining
* The statement is a lazy construct
* Need to later execute
    * Makes them reusable!

#### a) Create the statement

In [56]:
from sqlalchemy import select
stmt = select('*').select_from(Hero)

#### b) Execute the statement

In [57]:
session.execute(stmt).fetchmany(5)

[]

## The core of a SQL statement

* All `SQL` statements have the form `SELECT .... FROM ...`
* We can add other clauses, for example a filter
* Notice we don't need the `select_from` due to using `Hero` in select

In [44]:
stmt2 = select([Hero.c.name, Hero.c.gender, Hero.c.eye_color]).where(Hero.c.gender == 'Male')
session.execute(stmt2).fetchmany(5)

[('A-Bomb', 'Male', 'yellow'),
 ('Abe Sapien', 'Male', 'blue'),
 ('Abin Sur', 'Male', 'blue'),
 ('Abomination', 'Male', 'green'),
 ('Abraxas', 'Male', 'blue')]

## Grouping and counting

We can add `group_by` and `count` to aggregate the data.

In [55]:
from sqlalchemy import func as f
stmt3 = select([Hero.c.gender, 
                f.count(Hero.c.gender).label('gender_count')]).\
          group_by(Hero.c.gender)
session.execute(stmt3).fetchall()

[(None, 0), ('Female', 200), ('Male', 505)]

In [54]:
from sqlalchemy import func as f
stmt3 = select([Hero.c.gender, 
                f.avg(Hero.c.height).label('avg_height')]).\
         group_by(Hero.c.gender)
session.execute(stmt3).fetchall()

[(None, 177.06666666666666),
 ('Female', 174.68402777777777),
 ('Male', 191.97486033519553)]

## Summary - Database Communication

* `engine` "speaks" our db language
* `session` provides the API to query the db
* `session` $\leftrightarrow$ `engine` $\leftrightarrow$ `sqlite`

## Up Next

1. Now you should complete [Lab 1](./lab_1_super_powers_and_SQL.ipynb) to practice creating a `sqlalchemy` database, then
2. We will look at using `DataFrame`s in `pyspark` in [Lecture 1.3](./1_3_introduction_to_pyspark_dataframes.ipynb)
3. We will take a unified look queries in [Lecture 2.1](./2_1_dataframe_verbs_select_filter_mutate.ipynb)

# Appendix 1 - Declarative creation of `sqlalchemy` database tables

## An example `sqlalchemy` table

In [ ]:
# Start over by deleting the existing db
!rm tutorial.db

In [54]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

db = create_engine('sqlite:///tutorial.db')

db.echo = True

Base = declarative_base()

class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    password = Column(String)
    
    def __repr__(self):
        base = "User(name={0}, age={1}, password={2})"
        return base.format(self.name, self.age, self.password)

Base.metadata.create_all(db)

2019-01-18 11:40:49,851 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-18 11:40:49,857 INFO sqlalchemy.engine.base.Engine ()
2019-01-18 11:40:49,862 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-18 11:40:49,867 INFO sqlalchemy.engine.base.Engine ()
2019-01-18 11:40:49,873 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("user")
2019-01-18 11:40:49,875 INFO sqlalchemy.engine.base.Engine ()
2019-01-18 11:40:49,883 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	age INTEGER, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2019-01-18 11:40:49,885 INFO sqlalchemy.engine.base.Engine ()
2019-01-18 11:40:49,902 INFO sqlalchemy.engine.base.Engine COMMIT


## Summary - Database Representation

* `Base` class handles the `db` metadata
* All tables inherit `Base`
* `Column`s are class attributes on the tables
* `Column`s hold the type information

## The `Base` class

<img src="./img/db_repr_1.png" width=400>

## The `Used` inherits from `Base`

<img src="./img/db_repr_2.png" width=400>

## The `Column`s are assigned as class attributes

<img src="./img/db_repr_3.png" width=400>

## The `__repr__` properly displays each row

<img src="./img/db_repr_4.png" width=400>

## Step 1 - Connect to a database

`create_engine`

* connects to an existing db
* creates it if necessary
* speaks the database's language

In [36]:
db = create_engine('sqlite:///tutorial.db')

## To echo, or not to echo

* In this example, `db.echo = True`
* Normally, `db.echo = False`
    * SHUT UP ALREADY

In [37]:
db.echo = True

## Step 2 - Make a base class

* Our table will inherit from this class
* This class hold the metadata
    * metadata == data about our data

In [38]:
Base = declarative_base()

## Defining a class representation of a table

* Needs to match our table
* the `__table__` holds the metadata
* Will be instanciated later

In [55]:
User.__table__

Table('user', MetaData(bind=None), Column('id', Integer(), table=<user>, primary_key=True, nullable=False), Column('name', String(), table=<user>), Column('age', Integer(), table=<user>), Column('password', String(), table=<user>), schema=None)

## Base is a metaclass which creates a subclass

<img src="./img/base_metaclass.png" width=600>

## Making an instance of our table

Here the `db` engine is telling sqlite to create the table(s)

In [40]:
Base.metadata.create_all(db)

2019-01-18 11:33:53,697 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-18 11:33:53,698 INFO sqlalchemy.engine.base.Engine ()
2019-01-18 11:33:53,701 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-18 11:33:53,702 INFO sqlalchemy.engine.base.Engine ()


## Inspecting the Table - Inspecting Table Names

1. Create a instance of `user` with `Table`
2. Use a comprehension, as shown below.

In [56]:
from sqlalchemy import Table
user = Table('user', Base.metadata, autoload=True, autoload_with=db)
[col.name for col in user.columns]

['id', 'name', 'age', 'password']

## Tables are unique

* Only 1 instance of a given table.
* Handled by `sqlalchemy`
* New instances just point to the older one

In [57]:
user2 =  Table('user', Base.metadata)
user is user2

True

## Fine-grain Inspection with `inspect`

In [58]:
from sqlalchemy import inspect
insp = inspect(db)

## Getting the name of all tables

In [59]:
insp.get_table_names()

2019-01-18 11:41:20,153 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-01-18 11:41:20,154 INFO sqlalchemy.engine.base.Engine ()


['user']

## Getting detailed information on columns

In [60]:
insp.get_columns('user')

2019-01-18 11:41:24,102 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("user")
2019-01-18 11:41:24,103 INFO sqlalchemy.engine.base.Engine ()


[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'name',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'age',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'password',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

## More to come

We will revisit inspection when we set up table relationships.

# Adding Data to a Table

Next, we will look at adding and commiting data to a table.

## `sqlalchemy` is lazy

* You need to *tell it* to complete tasks
* Using `db.echo == True` allows us to see when things actually happen
* This is good!
* Double check your work before commiting!

## Making a session object

We make a `session` that allows us to interact with the `db`

#### a) Make a Session class bound to `db`

In [61]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db)

#### b) Make an instance of the `Session` class

In [62]:
session = Session()

## Summary - Database Communication

* `engine` "speaks" our db language
* `session` provides the API to change the db
* `session` $\leftrightarrow$ `engine` $\leftrightarrow$ `sqlite`

## Adding data to a table - one row at a time

Use keywords for each column

In [63]:
ed_user = User(name='ed', age=42, password='edspassword')
session.add(ed_user)

## Using a `dict` and keyword unpacking

* Information in a `dict`
* Unpack the info with `**`

In [64]:
beth_info=   {'name':'beth', 'age':39, 'password':'bethspassword'}
beth_user = User(**beth_info)
session.add(beth_user)

## Keyword unpacking illustrated

<img src="./img/keyword_unpacking_1.png" width=500>

## Keyword unpacking illustrated

<img src="./img/keyword_unpacking_2.png" width=500>

## Keyword unpacking illustrated

<img src="./img/keyword_unpacking_3.png" width=500>

## ... did you notice ...

* There was no echo
* `ed_user` lives in Python
* Not in the actual `db`
* We need to commit later

In [65]:
session.new

IdentitySet([User(name=ed, age=42, password=edspassword), User(name=beth, age=39, password=bethspassword)])

In [66]:
session.commit()

2019-01-18 11:41:55,338 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-18 11:41:55,342 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age, password) VALUES (?, ?, ?)
2019-01-18 11:41:55,346 INFO sqlalchemy.engine.base.Engine ('ed', 42, 'edspassword')
2019-01-18 11:41:55,352 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age, password) VALUES (?, ?, ?)
2019-01-18 11:41:55,356 INFO sqlalchemy.engine.base.Engine ('beth', 39, 'bethspassword')
2019-01-18 11:41:55,359 INFO sqlalchemy.engine.base.Engine COMMIT


## Adding many users at once

#### 1. Store data in a `dict`

In [67]:
rows = ({'name': 'John',  'age': 42 , 'password':'johns_password'},
        {'name': 'Susan', 'age': 57,  'password':'susan_password'},
        {'name': 'Carl',  'age': 33 , 'password':'carl_password'})

#### 2. Make instances of `User` for each

In [68]:
users = [User(**row) for row in rows]
users

[User(name=John, age=42, password=johns_password),
 User(name=Susan, age=57, password=susan_password),
 User(name=Carl, age=33, password=carl_password)]

#### 3. Add the users to the `db` using `session`

In [69]:
session.add_all(users)

In [70]:
session.new

IdentitySet([User(name=John, age=42, password=johns_password), User(name=Susan, age=57, password=susan_password), User(name=Carl, age=33, password=carl_password)])

In [71]:
session.commit()

2019-01-18 11:42:09,704 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age, password) VALUES (?, ?, ?)
2019-01-18 11:42:09,706 INFO sqlalchemy.engine.base.Engine ('John', 42, 'johns_password')
2019-01-18 11:42:09,711 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age, password) VALUES (?, ?, ?)
2019-01-18 11:42:09,715 INFO sqlalchemy.engine.base.Engine ('Susan', 57, 'susan_password')
2019-01-18 11:42:09,718 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age, password) VALUES (?, ?, ?)
2019-01-18 11:42:09,721 INFO sqlalchemy.engine.base.Engine ('Carl', 33, 'carl_password')
2019-01-18 11:42:09,723 INFO sqlalchemy.engine.base.Engine COMMIT


## Querying our `db` the Python way

In [72]:
session.query(User).all()

2019-01-18 11:42:14,032 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age, user.password AS user_password 
FROM user
2019-01-18 11:42:14,034 INFO sqlalchemy.engine.base.Engine ()


[User(name=ed, age=42, password=edspassword),
 User(name=beth, age=39, password=bethspassword),
 User(name=John, age=42, password=johns_password),
 User(name=Susan, age=57, password=susan_password),
 User(name=Carl, age=33, password=carl_password)]

## `sqlalchemy` can help you learn SQL

<img src="./img/learn_sql.png" width=400>

## Executing raw SQL

* SQL commands are Python `str` passed through `text`
* Use `execute`

In [77]:
from sqlalchemy.sql import text
statement = text("""SELECT user.id AS user_id, user.name AS user_name, 
                    user.age AS user_age, user.password AS user_password 
                    FROM user""")
result = session.execute(statement).fetchall()
result

2019-01-18 11:57:31,719 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, 
                    user.age AS user_age, user.password AS user_password 
                    FROM user
2019-01-18 11:57:31,723 INFO sqlalchemy.engine.base.Engine ()


[(1, 'ed', 42, 'edspassword'),
 (2, 'beth', 39, 'bethspassword'),
 (3, 'John', 42, 'johns_password'),
 (4, 'Susan', 57, 'susan_password'),
 (5, 'Carl', 33, 'carl_password')]

## Did you notice?

<img src="./img/fetchall.png" width=400>

`fetchall` returned Python data

## Applying a filter `db` the Python way

In [78]:
session.query(User).filter(User.age > 40).all()

2019-01-18 11:57:58,344 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age, user.password AS user_password 
FROM user 
WHERE user.age > ?
2019-01-18 11:57:58,347 INFO sqlalchemy.engine.base.Engine (40,)


[User(name=ed, age=42, password=edspassword),
 User(name=John, age=42, password=johns_password),
 User(name=Susan, age=57, password=susan_password)]

## Inspect the SQL on the last query

<img src="./img/sql_parameter.png" width=400>

## Executing raw SQL with a parameter

* Give the parameter a name
* Put `:` before the name
* Use `text.param` to assign a value

In [79]:
stmt = text("""SELECT user.id AS user_id, 
                      user.name AS user_name, 
                      user.age AS user_age, 
                      user.password AS user_password 
               FROM user 
               WHERE user.age > :limit""")
result = session.execute(stmt.params(limit = 40)).fetchall()
result

2019-01-18 11:58:34,702 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, 
                      user.name AS user_name, 
                      user.age AS user_age, 
                      user.password AS user_password 
               FROM user 
               WHERE user.age > ?
2019-01-18 11:58:34,704 INFO sqlalchemy.engine.base.Engine (40,)


[(1, 'ed', 42, 'edspassword'),
 (3, 'John', 42, 'johns_password'),
 (4, 'Susan', 57, 'susan_password')]

## Using parameters make your code reusable

In [80]:
result = session.execute(stmt.params(limit = 45)).fetchall()
result

2019-01-18 11:58:39,853 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, 
                      user.name AS user_name, 
                      user.age AS user_age, 
                      user.password AS user_password 
               FROM user 
               WHERE user.age > ?
2019-01-18 11:58:39,855 INFO sqlalchemy.engine.base.Engine (45,)


[(4, 'Susan', 57, 'susan_password')]

In [39]:
engine.execute("SELECT * FROM heroes").fetchmany(5)

[(0, 0, 'A-Bomb', 'Male', 'yellow', 'Human', 'No Hair', 203.0, 'Marvel Comics', None, 'good', 441.0),
 (1, 1, 'Abe Sapien', 'Male', 'blue', 'Icthyo Sapien', 'No Hair', 191.0, 'Dark Horse Comics', 'blue', 'good', 65.0),
 (2, 2, 'Abin Sur', 'Male', 'blue', 'Ungaran', 'No Hair', 185.0, 'DC Comics', 'red', 'good', 90.0),
 (3, 3, 'Abomination', 'Male', 'green', 'Human / Radiation', 'No Hair', 203.0, 'Marvel Comics', None, 'bad', 441.0),
 (4, 4, 'Abraxas', 'Male', 'blue', 'Cosmic Entity', 'Black', None, 'Marvel Comics', None, 'bad', None)]

In [34]:
from toolz import first

new_names = lambda df: {clean_names(old_name):old_name for old_name in df.columns}
fix_all_names = lambda df: (df >> rename(**new_names(df)))

df_iter = pd.read_csv('./data/heroes_information.csv', 
                      na_values=['-', '', '-99.0'],
                      chunksize=100)
first_chunk = first(df_iter)
first_chunk_renamed = fix_all_names(first_chunk)
first_chunk_renamed.to_sql('heroes', 
                           con=engine, 
                           dtype=sql_types, 
                           if_exists='replace')
for chunk in df_iter:
    chunk_renamed = fix_all_names(chunk)
    chunk_renamed.to_sql('heroes', 
                         con=engine, 
                         dtype=sql_types, 
                         if_exists='append')

In [36]:
engine.execute("SELECT * FROM heroes").fetchmany(5)

[(0, 0, 'A-Bomb', 'Male', 'yellow', 'Human', 'No Hair', 203.0, 'Marvel Comics', None, 'good', 441.0),
 (1, 1, 'Abe Sapien', 'Male', 'blue', 'Icthyo Sapien', 'No Hair', 191.0, 'Dark Horse Comics', 'blue', 'good', 65.0),
 (2, 2, 'Abin Sur', 'Male', 'blue', 'Ungaran', 'No Hair', 185.0, 'DC Comics', 'red', 'good', 90.0),
 (3, 3, 'Abomination', 'Male', 'green', 'Human / Radiation', 'No Hair', 203.0, 'Marvel Comics', None, 'bad', 441.0),
 (4, 4, 'Abraxas', 'Male', 'blue', 'Cosmic Entity', 'Black', None, 'Marvel Comics', None, 'bad', None)]

## Reading with `csv.DictReader`

In [85]:
from csv import DictReader, Sniffer

with open('./data/heroes_information.csv') as csvfile:
    dialect = Sniffer().sniff(csvfile.read(50))
    csvfile.seek(0)
    reader = DictReader(csvfile, dialect=dialect)
    columns = reader.fieldnames
    rows = [row for row in reader]

## Deconstruct the last code

<img src="./img/dict_reader.png" width=500>

## `columns` is a list of column names

In [89]:
columns

['Id',
 'name',
 'Gender',
 'Eye color',
 'Race',
 'Hair color',
 'Height',
 'Publisher',
 'Skin color',
 'Alignment',
 'Weight']

## Rows are `dict` with `key = col_name` and `val = cell value`

In [88]:
rows[0]

OrderedDict([('Id', '0'),
             ('name', 'A-Bomb'),
             ('Gender', 'Male'),
             ('Eye color', 'yellow'),
             ('Race', 'Human'),
             ('Hair color', 'No Hair'),
             ('Height', '203.0'),
             ('Publisher', 'Marvel Comics'),
             ('Skin color', '-'),
             ('Alignment', 'good'),
             ('Weight', '441.0')])

## TODO list

* Fix the column names
* Replace `'-'` and `''` with `None`
* Convert height and weight to `float`s

## Fix the column names

* make lower-case
* strip whitespace
* replace inner spaces with `_`

In [90]:
rows_clean_lbl = [{lbl.lower().strip().replace(' ', '_'):val for lbl, val in row.items()} for row in rows]
rows_clean_lbl[:2]

[{'id': '0',
  'name': 'A-Bomb',
  'gender': 'Male',
  'eye_color': 'yellow',
  'race': 'Human',
  'hair_color': 'No Hair',
  'height': '203.0',
  'publisher': 'Marvel Comics',
  'skin_color': '-',
  'alignment': 'good',
  'weight': '441.0'},
 {'id': '1',
  'name': 'Abe Sapien',
  'gender': 'Male',
  'eye_color': 'blue',
  'race': 'Icthyo Sapien',
  'hair_color': 'No Hair',
  'height': '191.0',
  'publisher': 'Dark Horse Comics',
  'skin_color': 'blue',
  'alignment': 'good',
  'weight': '65.0'}]

## STINKY CABBAGE!

The last expression processed two levels of abstraction.

* Rows consists of row `dict`
* A row consists of `(lbl, val)` pairs

**Clean code rule:** Write one function per level of abstraction

## Review - Code Smell

<img src="https://imgs.xkcd.com/comics/code_quality.png">

<img src="./img/dictreader_levels.png" width=600>

## Review - Refactoring code

* Work inside-out
* Put expressions in `lambda` functions
* Replace code with function calls
* Repeat

## Step 0 - Take a big whiff

<img src="./img/refactor_1.png" width=800>

## Step 1 - Create a value/label function

<img src="./img/refactor_2.png" width=800>

## Step 2 - Replace value/label expression with a function call

<img src="./img/refactor_3.png" width=800>

## Step 3 - Create a row function

<img src="./img/refactor_4.png" width=800>

## Step 4 - Replace row expression with a function call

<img src="./img/refactor_5.png" width=800>

## Result - One function for each level of abstraction

*CLEAN CODE!*

<img src="./img/refactor_6.png" width=800>

In [91]:
clean_label = lambda lbl: lbl.lower().strip().replace(' ', '_')
clean_lbl_row = lambda row: {clean_label(lbl):val for lbl, val in row.items()}
clean_lbl_rows = lambda row: [clean_lbl_row(row) for row in rows]
rows_clean_lbl = clean_lbl_rows(rows)
rows_clean_lbl[:2]

[{'id': '0',
  'name': 'A-Bomb',
  'gender': 'Male',
  'eye_color': 'yellow',
  'race': 'Human',
  'hair_color': 'No Hair',
  'height': '203.0',
  'publisher': 'Marvel Comics',
  'skin_color': '-',
  'alignment': 'good',
  'weight': '441.0'},
 {'id': '1',
  'name': 'Abe Sapien',
  'gender': 'Male',
  'eye_color': 'blue',
  'race': 'Icthyo Sapien',
  'hair_color': 'No Hair',
  'height': '191.0',
  'publisher': 'Dark Horse Comics',
  'skin_color': 'blue',
  'alignment': 'good',
  'weight': '65.0'}]

## Replace `"-"` and `''` with `None`

**Remember:** Rows contain Row contain `(lbl, val)` pairs

In [92]:
# Value function
clean_missing = lambda val: None if val == '-' or val =='' else val
# Row function
clean_missing_row = lambda row: {lbl:clean_missing(val) for lbl, val in row.items()}
# Table function
clean_missing_rows = lambda rows: [clean_missing_row(row) for row in rows]

In [93]:
rows_clean_missing = clean_missing_rows(rows_clean_lbl)
rows_clean_missing[3]

{'id': '3',
 'name': 'Abomination',
 'gender': 'Male',
 'eye_color': 'green',
 'race': 'Human / Radiation',
 'hair_color': 'No Hair',
 'height': '203.0',
 'publisher': 'Marvel Comics',
 'skin_color': None,
 'alignment': 'bad',
 'weight': '441.0'}

## <font color="red"> Exercise 2 </font>
    
**Task:** Write the functions needed to convert height and weight to floats.  Obey the clean-code rule and use only `lambda` functions.  **Hint:** the inner-most function will need both the `lbl` and `val` and an `if` expression

In [96]:
maybe_float  = lambda val: None if not val else float(val) 
convert_if_needed = lambda lbl, val: maybe_float(val) if lbl in ('height', 'weight') else val
convert_row_to_float = lambda row: {lbl:convert_if_needed(lbl, val) for lbl, val in row.items()}
convert_rows_to_float = lambda rows: [convert_row_to_float(row) for row in rows]
rows_totally_clean = convert_rows_to_float(rows_clean_missing) 
rows_totally_clean[:2]

[{'id': '0',
  'name': 'A-Bomb',
  'gender': 'Male',
  'eye_color': 'yellow',
  'race': 'Human',
  'hair_color': 'No Hair',
  'height': 203.0,
  'publisher': 'Marvel Comics',
  'skin_color': None,
  'alignment': 'good',
  'weight': 441.0},
 {'id': '1',
  'name': 'Abe Sapien',
  'gender': 'Male',
  'eye_color': 'blue',
  'race': 'Icthyo Sapien',
  'hair_color': 'No Hair',
  'height': 191.0,
  'publisher': 'Dark Horse Comics',
  'skin_color': 'blue',
  'alignment': 'good',
  'weight': 65.0}]

# Defining a `sqlalchemy` table

Now that the data is clean, we define a SQL table

## Importing `sqllite`

In [103]:
import sqlalchemy
sqlalchemy.__version__ 

'1.1.13'

## Set Up

In [121]:
!rm heroes_new.db

In [122]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///heroes_new.db', echo=True)

Base = declarative_base()

## Table class definition

In [123]:
class Hero(Base):
    __tablename__ = 'heroes'

    id          = Column(Integer, primary_key=True)
    name        = Column(String)
    gender      = Column(String)
    eye_color   = Column(String)
    race        = Column(String)
    hair_color  = Column(String)
    height      = Column(Float)
    publisher   = Column(String)
    skin_color  = Column(String)
    alignment   = Column(String)
    weight      = Column(Float)

    def __repr__(self):
       base = ('Hero(id={0}, name={1}, gender={2}, eye_color={3},\n' + 
               '\trace={4}, hair_color={5}, height={6}, publisher={7},\n' + 
               '\tskin_color={8}, alignment={9}, weight={10})')
       return base.format(self.id          ,
                          self.name        ,
                          self.gender      ,
                          self.eye_color   ,
                          self.race        ,
                          self.hair_color  ,
                          self.height      ,
                          self.publisher   ,
                          self.skin_color  ,
                          self.alignment   ,
                          self.weight)

Base.metadata.create_all(engine)

2019-01-18 12:21:14,928 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-18 12:21:14,930 INFO sqlalchemy.engine.base.Engine ()
2019-01-18 12:21:14,934 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-18 12:21:14,936 INFO sqlalchemy.engine.base.Engine ()
2019-01-18 12:21:14,939 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("heroes")
2019-01-18 12:21:14,941 INFO sqlalchemy.engine.base.Engine ()
2019-01-18 12:21:14,944 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE heroes (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	gender VARCHAR, 
	eye_color VARCHAR, 
	race VARCHAR, 
	hair_color VARCHAR, 
	height FLOAT, 
	publisher VARCHAR, 
	skin_color VARCHAR, 
	alignment VARCHAR, 
	weight FLOAT, 
	PRIMARY KEY (id)
)


2019-01-18 12:21:14,947 INFO sqlalchemy.engine.base.Engine ()
2019-01-18 12:21:14,959 INFO sqlalchemy.engine.base.Engine COMMIT


## Populating a list of rows

In [124]:
heroes = [Hero(**r) for r in rows_totally_clean]
heroes[:2]

[Hero(id=0, name=A-Bomb, gender=Male, eye_color=yellow,
 	race=Human, hair_color=No Hair, height=203.0, publisher=Marvel Comics,
 	skin_color=None, alignment=good, weight=441.0),
 Hero(id=1, name=Abe Sapien, gender=Male, eye_color=blue,
 	race=Icthyo Sapien, hair_color=No Hair, height=191.0, publisher=Dark Horse Comics,
 	skin_color=blue, alignment=good, weight=65.0)]

## Connecting a session to the engine/db

In [125]:
Session = sessionmaker(bind=engine)
session = Session()

## Added and committing the rows

In [126]:
session.add_all(heroes)

In [136]:
session.commit()

2019-01-18 13:08:23,429 INFO sqlalchemy.engine.base.Engine COMMIT


## Getting the first five rows

In [131]:
first_five = session.query(Hero).slice(0, 5).all()
first_five

2019-01-18 12:23:30,314 INFO sqlalchemy.engine.base.Engine SELECT heroes.id AS heroes_id, heroes.name AS heroes_name, heroes.gender AS heroes_gender, heroes.eye_color AS heroes_eye_color, heroes.race AS heroes_race, heroes.hair_color AS heroes_hair_color, heroes.height AS heroes_height, heroes.publisher AS heroes_publisher, heroes.skin_color AS heroes_skin_color, heroes.alignment AS heroes_alignment, heroes.weight AS heroes_weight 
FROM heroes
 LIMIT ? OFFSET ?
2019-01-18 12:23:30,316 INFO sqlalchemy.engine.base.Engine (5, 0)


[Hero(id=0, name=A-Bomb, gender=Male, eye_color=yellow,
 	race=Human, hair_color=No Hair, height=203.0, publisher=Marvel Comics,
 	skin_color=None, alignment=good, weight=441.0),
 Hero(id=1, name=Abe Sapien, gender=Male, eye_color=blue,
 	race=Icthyo Sapien, hair_color=No Hair, height=191.0, publisher=Dark Horse Comics,
 	skin_color=blue, alignment=good, weight=65.0),
 Hero(id=2, name=Abin Sur, gender=Male, eye_color=blue,
 	race=Ungaran, hair_color=No Hair, height=185.0, publisher=DC Comics,
 	skin_color=red, alignment=good, weight=90.0),
 Hero(id=3, name=Abomination, gender=Male, eye_color=green,
 	race=Human / Radiation, hair_color=No Hair, height=203.0, publisher=Marvel Comics,
 	skin_color=None, alignment=bad, weight=441.0),
 Hero(id=4, name=Abraxas, gender=Male, eye_color=blue,
 	race=Cosmic Entity, hair_color=Black, height=-99.0, publisher=Marvel Comics,
 	skin_color=None, alignment=bad, weight=-99.0)]

## Hiding these details

* Table setup will rarely change
* We can hide these static declarations in a module

## Making `my_heroes.py`

Copy the following code into a file (in the root repo directory) named `my_heroes.py`

In [122]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///heroes_new.db', echo=True)

Base = declarative_base()

class Hero(Base):
    __tablename__ = 'heroes'

    id          = Column(Integer, primary_key=True)
    name        = Column(String)
    gender      = Column(String)
    eye_color   = Column(String)
    race        = Column(String)
    hair_color  = Column(String)
    height      = Column(Float)
    publisher   = Column(String)
    skin_color  = Column(String)
    alignment   = Column(String)
    weight      = Column(Float)

    def __repr__(self):
       base = ('Hero(id={0}, name={1}, gender={2}, eye_color={3},\n' + 
               '\trace={4}, hair_color={5}, height={6}, publisher={7},\n' + 
               '\tskin_color={8}, alignment={9}, weight={10})')
       return base.format(self.id          ,
                          self.name        ,
                          self.gender      ,
                          self.eye_color   ,
                          self.race        ,
                          self.hair_color  ,
                          self.height      ,
                          self.publisher   ,
                          self.skin_color  ,
                          self.alignment   ,
                          self.weight)

Base.metadata.create_all(engine)

## Importing and working with a predefined db

We can now import `engine` and `Hero` from `my_heroes`

In [140]:
from my_heroes import engine, Hero
engine.echo = False
Session2 = sessionmaker(bind=engine)
session2 = Session2()
session2.query(Hero).filter(Hero.name.startswith('B')).filter(Hero.height > 195).all()

[Hero(id=59, name=Bane, gender=Male, eye_color=None,
 	race=Human, hair_color=None, height=203.0, publisher=DC Comics,
 	skin_color=None, alignment=bad, weight=180.0),
 Hero(id=71, name=Battlestar, gender=Male, eye_color=brown,
 	race=None, hair_color=Black, height=198.0, publisher=Marvel Comics,
 	skin_color=None, alignment=good, weight=133.0),
 Hero(id=78, name=Beta Ray Bill, gender=Male, eye_color=None,
 	race=None, hair_color=No Hair, height=201.0, publisher=Marvel Comics,
 	skin_color=None, alignment=good, weight=216.0),
 Hero(id=91, name=Bishop, gender=Male, eye_color=brown,
 	race=Mutant, hair_color=No Hair, height=198.0, publisher=Marvel Comics,
 	skin_color=None, alignment=good, weight=124.0),
 Hero(id=119, name=Bloodaxe, gender=Female, eye_color=blue,
 	race=Human, hair_color=Brown, height=218.0, publisher=Marvel Comics,
 	skin_color=None, alignment=bad, weight=495.0),
 Hero(id=131, name=Booster Gold, gender=Male, eye_color=blue,
 	race=Human, hair_color=Blond, height=196.0, 

## Up Next

Next, you should complete [Lab 1](./lab_1_super_powers_and_SQL.ipynb), which involves building and querying a database of super hero powers.